Gerekli python kütüphanelerini içe aktararak işe başlıyoruz.


In [1]:
import cv2 as cv

Modelin hazır dosyalarını kullanarak opencv yardımı ile mask-rcnn modelini oluşturuyoruz.

In [2]:
model = cv.dnn.readNetFromTensorflow("dnn/frozen_inference_graph_coco.pb",
	                                 "dnn/mask_rcnn_inception_v2_coco_2018_01_28.pbtxt")

Test için kullanacağımız resimi içe aktarıyor ve en ve boy ölçülerini alıyoruz.

In [3]:
img = cv.imread("test.jpg")
height, width = img.shape[:2]

2.satırda resimin en ve boy ölçüleri aldık, bunun yapılma sebebi resimi tespit için modele gönderirken boyu küçültülür ve bu şekilde modele gönderilir. Bunun sonucu olarakta bounding box'lar küçük resime göre oranlara sahip olur. Bizde bu aldığımız bilgiler ile büyük resime göre yeniden oranlayacağız.

Resimi blob formatına çeviriyor ve modelin içerisine gönderiyoruz.

In [4]:
blob = cv.dnn.blobFromImage(img, swapRB=True)
model.setInput(blob)

forward() komutu ile modelden çıktilarımızı alıyoruz. Parantez içerisindeki değerler sabittir. Mask-RCNN modelinden iki çıktı döner. Bunlardan ilki tespit kutucukları yani bounding boxlar iken ikincisi bu cisimlerin maskeleridir. Bu kısımda herhangi bir maskeleme yapmayacağımız için sadece bounding box değerlerini çektik.

In [5]:
boxes = model.forward(["detection_out_final", "detection_masks"])[0]

In [6]:
detection_count = boxes.shape[2]

In [7]:
for i in range(detection_count):
	box = boxes[0, 0, i]
	class_id = box[1]
	score = box[2]
	if score < 0.5:
		continue
		
	x = int(box[3] * width)
	y = int(box[4] * height)
	x2 = int(box[5] * width)
	y2 = int(box[6] * height)

	cv.rectangle(img, (x, y), (x2, y2), (255, 0, 0), 1)

Tüm işlemler sonladığına göre artık sonuçları göremke için resimi ekranda gösterebiliriz.

In [9]:
cv.imshow("Image", img)
cv.waitKey(0)